## RECOMENDAÇÃO DE FILMES <br>
<p> Fonte de dados: https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

In [ ]:
pip install recommenders

In [ ]:
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.pandas_df_utils import user_item_pairs
from recommenders.models.sar import SAR
from sklearn.model_selection import train_test_split
import pandas as pd


In [ ]:
# Carregar os dados
avaliacoes = pd.read_csv('ratings.csv', dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'}, nrows=1000000)


# Renomeia as variaveis
avaliacoes.rename(columns = {'userId':'ID_USUARIO','movieId':'ID_FILME','rating':'AVALIACAO'}, inplace = True)


In [ ]:
# Vamos pegar o ID_USUARIO somente de usuários que fizeram mais de x avaliações
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 10
y = qt_avaliacoes[qt_avaliacoes].index
y.shape


(8474,)

In [ ]:
# Pegando somente avaliacoes dos usuarios que avaliaram mais de x vezes
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(y)]
avaliacoes.shape

(990316, 4)

In [ ]:
# Divisão estratificada para garantir que os usuários do teste estejam no treinamento
train, test = train_test_split(avaliacoes, test_size=0.2, stratify=avaliacoes['ID_USUARIO'])

train.head()

,ID_USUARIO,ID_FILME,AVALIACAO,timestamp
14298,173,4886,3.0,1443176891
29991,346,4978,4.0,1023718726
830661,8539,56171,3.0,1477068132
395761,4160,98243,4.0,1354425826
780476,7983,382,4.0,965999633


In [ ]:
# Criar o modelo SAR (algoritmo de recomendação simples baseado em afinidade)
model = SAR(col_user='ID_USUARIO', col_item='ID_FILME', col_rating='AVALIACAO',
            similarity_type='jaccard', time_decay_coefficient=30,
            timedecay_formula=True)

# Treinar o modelo
model.fit(train)
train

,ID_USUARIO,ID_FILME,AVALIACAO,timestamp
14298,173,4886,3.0,1443176891
29991,346,4978,4.0,1023718726
830661,8539,56171,3.0,1477068132
395761,4160,98243,4.0,1354425826
780476,7983,382,4.0,965999633
...,...,...,...,...
797062,8216,2324,4.5,1348618055
204995,2163,5952,5.0,1477911637
607802,6137,1036,4.0,1288027589
303653,3226,1394,2.0,906209007


In [ ]:
# Fazer recomendações para um usuário específico
ID_USUARIO = 230
top_k = 10
recommendations = model.recommend_k_items(test, top_k=top_k, remove_seen=True)
user_recommendations = recommendations[recommendations['ID_USUARIO'] == ID_USUARIO]

print("Recomendações para o usuário:", user_recommendations)

Recomendações para o usuário:        ID_USUARIO  ID_FILME    prediction
35470         230      5902  1.869618e-35
35471         230      5878  1.824859e-35
35472         230      4848  1.752353e-35
35473         230      6953  1.692355e-35
35474         230      3556  1.670842e-35
35475         230      6711  1.599619e-35
35476         230      6870  1.575069e-35
35477         230     27773  1.569608e-35
35478         230      4881  1.551133e-35
35479         230      3910  1.548794e-35
